In [ ]:
!pip install bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 22.7 MB/s eta 0:00:00:00:0100:01


In [ ]:
print("hello")

hello


In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [ ]:
!pip install peft


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from transformers import DataCollatorForSeq2Seq
from transformers import DataCollatorForLanguageModeling

In [ ]:
dataset = load_dataset("cnn_dailymail", "3.0.0")

# Select only a subset (e.g., 5k training samples & 1k validation samples)
small_train = dataset["train"].shuffle(seed=42).select(range(5000))
small_valid = dataset["validation"].shuffle(seed=42).select(range(1000))

# Replace the dataset with the smaller subset
dataset["train"] = small_train
dataset["validation"] = small_valid


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
model_name = "gpt2"  # You can replace this with another model



In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(example):
    return tokenizer(example["article"], truncation=True, padding="max_length", max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets=tokenized_datasets.remove_columns(['article',"highlights"])
print(tokenized_datasets)
print(tokenized_datasets["train"][0]["input_ids"])


DatasetDict({
    train: Dataset({
        features: ['id', 'input_ids', 'attention_mask'],
        num_rows: 5000
    })
    validation: Dataset({
        features: ['id', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['id', 'input_ids', 'attention_mask'],
        num_rows: 11490
    })
})
[3886, 764, 9953, 12812, 764, 24676, 9148, 18422, 1961, 25, 764, 8753, 25, 3070, 17160, 11, 362, 2805, 2211, 764, 930, 764, 471, 49316, 25, 764, 8487, 25, 2998, 17160, 11, 362, 2805, 2211, 764, 7683, 1866, 286, 262, 976, 1641, 508, 3724, 287, 257, 9037, 44321, 422, 6588, 937, 28885, 22475, 561, 423, 587, 14917, 705, 33479, 2431, 3256, 10093, 531, 1909, 13, 383, 5920, 286, 6405, 3155, 1757, 290, 48498, 8261, 547, 5071, 7848, 511, 4957, 11, 6669, 49851, 11, 379, 262, 5175, 1363, 484, 4888, 319, 46656, 283, 293, 5995, 3250, 287, 7298, 13555, 11, 7421, 45294, 13, 383, 7949, 3558, 423, 783, 4721, 656, 262, 7040, 938, 3909, 11, 351, 10093, 2282, 262, 1

In [ ]:
tokenized_datasets.set_format("torch")
print(tokenized_datasets)
print(tokenized_datasets["train"][0]["input_ids"])

DatasetDict({
    train: Dataset({
        features: ['id', 'input_ids', 'attention_mask'],
        num_rows: 5000
    })
    validation: Dataset({
        features: ['id', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['id', 'input_ids', 'attention_mask'],
        num_rows: 11490
    })
})
tensor([ 3886,   764,  9953, 12812,   764, 24676,  9148, 18422,  1961,    25,
          764,  8753,    25,  3070, 17160,    11,   362,  2805,  2211,   764,
          930,   764,   471, 49316,    25,   764,  8487,    25,  2998, 17160,
           11,   362,  2805,  2211,   764,  7683,  1866,   286,   262,   976,
         1641,   508,  3724,   287,   257,  9037, 44321,   422,  6588,   937,
        28885, 22475,   561,   423,   587, 14917,   705, 33479,  2431,  3256,
        10093,   531,  1909,    13,   383,  5920,   286,  6405,  3155,  1757,
          290, 48498,  8261,   547,  5071,  7848,   511,  4957,    11,  6669,
        49851,    11,   379,  

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)


In [ ]:
#Apply LoRA
lora_config=LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["c_attn", "c_proj"],
    lora_dropout=0.05
)

In [ ]:
model=get_peft_model(model,lora_config)

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [ ]:
print(model)
# Check trainable parameters (After LoRA)
model.print_trainable_parameters()

PeftModel(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2SdpaAttention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D(nf=2304, nx=768)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (

In [ ]:
training_args=TrainingArguments(
    output_dir="./fine_tuned_llm",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=1,
    logging_dir="./logs",
    logging_steps=500,
    save_total_limit=2,
    report_to="none"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
#This collator automatically adds labels by shifting the input tokens during training.
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # mlm=True is for BERT-like models, but BLOOM is autoregressive
)

In [ ]:


trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

<ipython-input-20-de3f97fe3e75>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer=Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,3.540600,No log


TrainOutput(global_step=5000, training_loss=3.65288828125, metrics={'train_runtime': 351.5346, 'train_samples_per_second': 14.223, 'train_steps_per_second': 14.223, 'total_flos': 1318917242880000.0, 'train_loss': 3.65288828125, 'epoch': 1.0})

In [ ]:
trainer.save_model("./fine_tuned_llm")
tokenizer.save_pretrained("./fine_tuned_llm")

('./fine_tuned_llm/tokenizer_config.json',
 './fine_tuned_llm/special_tokens_map.json',
 './fine_tuned_llm/vocab.json',
 './fine_tuned_llm/merges.txt',
 './fine_tuned_llm/added_tokens.json',
 './fine_tuned_llm/tokenizer.json')

In [ ]:
model_path="/kaggle/working/fine_tuned_llm"
tokenizer=AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token  # Fix padding issue
model=AutoModelForCausalLM.from_pretrained(model_path,torch_dtype=torch.float16,device_map="auto")

In [ ]:
def generate_text(prompt,max_lenght=100):
    inputs=tokenizer(prompt,return_tensors="pt").to("cuda")
    with torch.no_grad():
        output=model.generate(
            **inputs,
            max_length=512,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,
            temperature=0.9,
            top_k=40,
            top_p=0.85,
            repetition_penalty=1.2
        )
    return tokenizer.decode(output[0],skip_special_tokens=True)

In [ ]:
prompt = "The latest advancements in AI suggest that"
generated_text = generate_text(prompt)
print("Generated Output:\n", generated_text)

Generated Output:
 The latest advancements in AI suggest that robots can be made to recognise and recognize smells. Robots are now capable of detecting smell's cumbersomeness as they walk around, for example through the scent hearers get from their eyes; noddles will detect a whistle on an armchair speaker with air-powered microphones but don't have these tools anymore . [ccordingto news reports , scientists were able develop machines which would stop sniffing when faced withdescription (	) - this is what you'd expect if humans went into practitionesive moods or challengies? In contrast we're still trying it here at MIT – robotic vision has been debated since 2008 by academics who describe them not just too much like dogs without any sense about where your ears stand: We've got tissues! This time though I'll tell some science fiction tale : The future looks brighter than ever before because my favourite thing was having two scissors fitted onto each other while talkingensical sentences